In [8]:
import os
import librosa
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

# Function to extract audio features (Mel-frequency cepstral coefficients - MFCCs)

dataset_directory = r'D:\Forbmax User Data\waqar sahi\Dataset\SD'
import os
import librosa
import tensorflow as tf
import numpy as np

# Load the VGGish model (you can replace this with your desired model)
vggish_model = tf.keras.applications.VGGish(include_top=False, weights='vggish_model_weights.h5', input_shape=(96, 64, 1))

# Define a function to extract embeddings
def extract_embeddings(audio_file):
    # Load and preprocess the audio
    audio, sr = librosa.load(audio_file, sr=vggish_model.input_shape[1])
    audio = audio.reshape(1, -1, 1)
    
    # Extract embeddings
    embeddings = vggish_model.predict(audio)
    
    return embeddings

# Path to your dataset folder
dataset_folder = dataset_directory

# Iterate through each folder (class) in the dataset
for label in os.listdir(dataset_folder):
    label_folder = os.path.join(dataset_folder, label)
    
    # Iterate through audio files in the folder
    for audio_file in os.listdir(label_folder):
        if audio_file.endswith('.wav'):  # Adjust file extension as needed
            audio_path = os.path.join(label_folder, audio_file)
            
            # Extract embeddings
            embeddings = extract_embeddings(audio_path)
            
            # Save embeddings with label
            save_path = os.path.join('embeddings', label)
            os.makedirs(save_path, exist_ok=True)
            np.save(os.path.join(save_path, f'{audio_file[:-4]}.npy'), embeddings)


AttributeError: module 'keras.api._v2.keras.applications' has no attribute 'VGGish'

In [7]:
import os
import librosa
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import euclidean_distances

# Function to extract audio features using YAMNet
def extract_features(audio_path):
    model = hub.load("https://tfhub.dev/google/yamnet/2")
    audio, sr = librosa.load(audio_path, sr=None)
    # Ensure the audio is in the range [-1.0, 1.0] as expected by YAMNet
    audio = audio / np.max(np.abs(audio))
    
    # Extract embeddings using YAMNet
    scores, embeddings, spectrogram = model(audio)
    
    return embeddings

# Function to load and preprocess audio samples
def load_and_preprocess_audio(audio_path):
    features = extract_features(audio_path)
    # You can perform further preprocessing here if needed
    return features

# Function to calculate Euclidean distance between two feature vectors
def calculate_distance(feature_vector1, feature_vector2):
    return np.linalg.norm(feature_vector1 - feature_vector2)

# Corrected dataset path with double backslashes
dataset_directory = r'D:\Forbmax User Data\waqar sahi\Dataset\SD'

# List all audio files in the dataset directory
audio_files = [os.path.join(root, file) for root, _, files in os.walk(dataset_directory) for file in files]

# Split audio files into a training set and a testing set (adjust as needed)
split_ratio = 0.8
num_train_samples = int(len(audio_files) * split_ratio)
train_audio_files = audio_files[:num_train_samples]
test_audio_files = audio_files[num_train_samples:]

# Extract features and create embeddings for training data
train_embeddings = {}  # Dictionary to store embeddings

for audio_path in train_audio_files:
    features = load_and_preprocess_audio(audio_path)
    person_name = os.path.basename(os.path.dirname(audio_path))
    
    if person_name not in train_embeddings:
        train_embeddings[person_name] = []
    train_embeddings[person_name].append(features)

# Testing the embeddings on the testing set
for test_audio_path in test_audio_files:
    test_features = load_and_preprocess_audio(test_audio_path)
    min_distance = float('inf')
    predicted_person = None
    
    for person_name, person_embeddings in train_embeddings.items():
        for train_features in person_embeddings:
            distance = calculate_distance(test_features, train_features)
            if distance < min_distance:
                min_distance = distance
                predicted_person = person_name
    
    print(f"Predicted Person: {predicted_person} (Min Distance: {min_distance:.2f})")


HTTPError: HTTP Error 404: Not Found